In [80]:
!pip install PyPokerEngine

In [0]:
import pypokerengine
from pypokerengine.api.game import setup_config, start_poker
import random
from pypokerengine.players import BasePokerPlayer


class FishPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [0]:
config = setup_config(max_round=100, initial_stack=1000, small_blind_amount=20)
config.register_player(name="f1", algorithm=FishPlayer())
config.register_player(name="f2", algorithm=FishPlayer())
config.register_player(name="f3", algorithm=FishPlayer())
config.register_player(name="f4", algorithm=FishPlayer())
#game_result = start_poker(config, verbose=1)


In [83]:
game_result

{'players': [{'name': 'fish_player',
   'stack': 175,
   'state': 'participating',
   'uuid': 'ooxbgnzaqhetdaxlkursgn'},
  {'name': 'human_player',
   'stack': 25,
   'state': 'participating',
   'uuid': 'nbfhvzrhbkcblpcbtikmsv'}],
 'rule': {'ante': 0,
  'blind_structure': {},
  'initial_stack': 100,
  'max_round': 10,
  'small_blind_amount': 5}}

In [0]:
#Jugador Consola

import pypokerengine.utils.visualize_utils as U

class ConsolePlayer(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state):
        print(U.visualize_declare_action(valid_actions, hole_card, round_state, self.uuid))
        action, amount = self._receive_action_from_console(valid_actions)
        return action, amount

    def receive_game_start_message(self, game_info):
        print(U.visualize_game_start(game_info, self.uuid))
        self._wait_until_input()

    def receive_round_start_message(self, round_count, hole_card, seats):
        print(U.visualize_round_start(round_count, hole_card, seats, self.uuid))
        self._wait_until_input()

    def receive_street_start_message(self, street, round_state):
        print(U.visualize_street_start(street, round_state, self.uuid))
        self._wait_until_input()

    def receive_game_update_message(self, new_action, round_state):
        print(U.visualize_game_update(new_action, round_state, self.uuid))
        self._wait_until_input()

    def receive_round_result_message(self, winners, hand_info, round_state):
        print(U.visualize_round_result(winners, hand_info, round_state, self.uuid))
        self._wait_until_input()

    def _wait_until_input(self):
        print("Enter some key to continue ...")

    # FIXME: This code would be crash if receives invalid input.
    #        So you should add error handling properly.
    def _receive_action_from_console(self, valid_actions):
        #action = raw_input("Enter action to declare >> ")
        action = 'raise' #@param ["'fold'", "'call'", "'raise'"] {type:"raw"}
        if action == 'fold': 
          amount = 0
        if action == 'call':  
          amount = valid_actions[1]['action']
        if action == 'raise':  
          amount = 5 #@param {type:"raw"}
        return action, amount

In [85]:
#Juega una partida
from pypokerengine.api.game import setup_config, start_poker

config = setup_config(max_round=10, initial_stack=100, small_blind_amount=5)
config.register_player(name="fish_player", algorithm=FishPlayer())
config.register_player(name="human_player", algorithm=ConsolePlayer())
game_result = start_poker(config, verbose=0)  # verbose=0 because game progress is visualized by ConsolePlayer

-- Game start (UUID = sfirwxcotwnwtajsoegtfx) --
-- rule --
  - 2 players game
  - 10 round
  - start stack = 100
  -        ante = 0
  - small blind = 5
Enter some key to continue ...
-- Round 1 start (UUID = sfirwxcotwnwtajsoegtfx) --
-- hole card --
  - ['CK', 'SA']
-- players information --
  - 0 : fish_player (kdqpfiacdczkjrhpeiwnal) => state : participating, stack : 90
  - 1 : human_player (sfirwxcotwnwtajsoegtfx) => state : participating, stack : 95
Enter some key to continue ...
-- New street start (UUID = sfirwxcotwnwtajsoegtfx) --
-- street --
  - preflop
Enter some key to continue ...
-- Declare your action (UUID = sfirwxcotwnwtajsoegtfx) --
-- valid actions --
  - fold
  - call: 10
  - raise: [15, 100]
-- hole card --
  - ['CK', 'SA']
-- round state --
  - dealer btn : fish_player
  - street : preflop
  - community card : []
  - pot : main = 15, side = []
  - players information
    - 0 : fish_player (kdqpfiacdczkjrhpeiwnal) => state : participating, stack : 90 <= BB
    - 

In [0]:
nb_simulation = 1000
nb_player = 3
hole_card = ['H4', 'D7']
community_card = ['D3', 'C5', 'C6']

def estimate_hand_strength(nb_simulation, nb_player, hole_card, community_card):
    simulation_results = []
    for i in range(nb_simulation):
        opponents_cards = []
        for j in range(nb_player-1):  # nb_opponents = nb_player - 1
            opponents_cards.append(draw_cards_from_deck(num=2))
        nb_need_community = 5 - len(community_card)
        community_card.append(draw_cards_from_deck(num=nb_need_community))
        result = observe_game_result(hole_card, community_card, opponents_cards)  # return 1 if win else 0
        simulation_results.append(result)
    average_win_rate = 1.0 * sum(simulation_results) / len(simulation_results)
    return average_win_rate

In [93]:
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
hole_card = gen_cards(['H4', 'D7'])
community_card = gen_cards(['D3', 'C5', 'C6'])
estimate_hole_card_win_rate(nb_simulation=1000, nb_player=3, hole_card=hole_card, community_card=community_card)
#estimate_hole_card_win_rate(nb_simulation=1000, nb_player=3, hole_card=hole_card, community_card=community_card)

0.824

In [0]:
#config.register_player(name="f5", algorithm=RandomPlayer())
#config.register_player(name="r1", algorithm=RandomPlayer())
#config.register_player(name="r2", algorithm=RandomPlayer())
#config.register_player(name="r3", algorithm=RandomPlayer())
#config.register_player(name="r4", algorithm=RandomPlayer())
#config.register_player(name="r5", algorithm=RandomPlayer())
#game_result = start_poker(config, verbose=1)